In [ ]:
from libs.corpus import openConllu, check_projectivity
import pyconll
import pyconll.util


In [ ]:


 
train = pyconll.load_from_file('data/it_isdt-ud-dev.conllu')
train_prepocesed=[]
for i,sent in enumerate(train):        
    sentence_preprocesed=[]
    for j,token in enumerate(sent):
        if(token.head is not None):
            sentence_preprocesed.append(token)
    train_prepocesed.append(sentence_preprocesed)

for i,sent in enumerate(train_prepocesed):     
    for j,token in enumerate(sent):
        print(token.head)
    print("------")
 